In [ ]:
# !python3 -m spacy download xx_ent_wiki_sm

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
real = pd.read_excel('data/data.xlsx')
# Меняем названия на более приемлемые:

target_shops = [
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"', 'ООО "АГРОТОРГ"', 'ООО "ЛЕНТА"',
   'АО "ДИКСИ ЮГ"', 'ООО "АШАН"', 'АО "ТАНДЕР"', 'ООО "БИЛЛА"', 'ООО "О`КЕЙ"',
    'ООО "АГРОАСПЕКТ"', 'ООО "АТАК"'
]

shops_rename = {
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"': 'ПЕРЕКРЕСТОК',
    'ООО "АГРОТОРГ"': 'ПЯТЕРОЧКА',
    'ООО "ЛЕНТА"': 'ЛЕНТА',
    'АО "ДИКСИ ЮГ"': 'ДИКСИ',
    'ООО "АШАН"': 'АШАН',
    'АО "ТАНДЕР"': 'МАГНИТ',
    'ООО "БИЛЛА"': 'БИЛЛА',
    'ООО "О`КЕЙ"': 'О`КЕЙ',
    'ООО "АГРОАСПЕКТ"': 'ПЯТЕРОЧКА',
    'ООО "АТАК"': 'АТАК'
}

real = real[real['shop_name'].isin(target_shops)].copy()
real['shop_name'] = real['shop_name'].apply(lambda x: shops_rename[x])
real = real.drop_duplicates(subset=['name'])
real.sample(2)

,shop_name,name,quantity,price,sum
62871,ЛЕНТА,Мята свежая вес 1кг,0.214,1999.99,428.00
73817,О`КЕЙ,Гель чистящий д/ унитаза Bref,1.000,87.99,87.99


In [3]:
real.head()

,shop_name,name,quantity,price,sum
19,ДИКСИ,МИНТАЙ ФИЛЕ Б/К СВ/МОР П/ПАК 8,1.0,199.90,199.90
20,ДИКСИ,БЗМЖ МОЛОКО СТРАНА ВАСИЛЬКИ У/,1.0,49.99,49.99
21,ДИКСИ,СУШКИ ТАРАЛЛИНИ С ЧЕСНОКОМ 180,1.0,29.99,29.99
22,ДИКСИ,ЯЙЦО КУРИНОЕ СТОЛОВОЕ 1КАТЕГОР,1.0,63.99,63.99
23,ДИКСИ,ПЕЧЕНЬЕ ЮБИЛЕЙНОЕ ВИТАМИНИЗИРО,2.0,29.99,59.98


In [4]:
clean = pd.read_csv('clean_data/all_clean.csv')

In [5]:
tmp = pd.read_csv('benchmarks/standard.csv')

In [6]:
def find_weight_or_volume(name: str)-> str:
    """Находит вес или объём товара."""

    pattern1 = r'\d+((\.|\,|x|X|х|Х)?)\d*([а-я]|[А-Я])*$' # ищет вес в конце
    pattern2 = r'\d+\w+' #  ищет вес в середине
    pattern3 = r'(\d+ кг)|(\d+ г)'  # находит `1 кг` или `1 г`
    result = re.search(pattern1, name)
    if result:
        return result.group(0)
    result = re.search(pattern2, name)
    if result:
        return result.group(0)
    result = re.search(pattern3, name)
    return result.group(0) if result else ''

In [7]:
# tmp['Название'].apply(find_weight_or_volume)

In [8]:
tmp['Название'] = tmp['Название'].str.lower()

In [41]:
from dicts import (
    PRODUCTS,
    BRANDS,
    SLASH_BRANDS,
    SLASH_PRODUCTS,
    BRANDS_WITH_NUMBERS
)

class NormalizeNames:
    def __init__(self, name: pd.Series):
        self.df = pd.DataFrame(name.copy()).rename(columns={'name': 'name_norm'})
        self.df['product_norm'] = None
        self.df['brand_norm'] = None
        
        self.blacklist = pd.read_csv('data/blacklist.csv')['0'].values
        self.brands = pd.read_csv('clean_data/brands_en.csv')['brand'].values
        
    @staticmethod
    def remove_all_words_which_contains_number(name, product, brand):
        
        # Find brands with numbers:
        for key, value in BRANDS_WITH_NUMBERS.items():
            if key in name:
                brand = value
                name = name.replace(key, '')
                break

        name = ' '.join(re.sub(r'\w*\d\w*', '', word).strip() for word in name.split())
        return pd.Series([name, product, brand])
    
    def remove_all_words_which_contains_slash(self, name, product, brand):
        for key, value in SLASH_BRANDS.items():
            if key in name:
                brand = value
                name = name.replace(key, ' ')
                break
        
        for key, value in SLASH_PRODUCTS.items():
            if key in name:
                product = value
                name = name.replace(key, ' ')
                break
        name = name.replace('/', ' ')
#         name = ' '.join(re.sub(r'\w*/\w*', ' ', word).strip() for word in name.split())
        return pd.Series([name, product, brand])
    
    def remove_punctuation_symbols(self, name, product, brand):
        for key, value in BRANDS.items():
            if key in name:
                brand = value
                name = name.replace(key, '')
                break
        
        tmp = [re.sub(r'([.,!?%:№*\(|\)])', ' ', word).strip() for word in name.split()]
        name = ' '.join(word[:-1] if word.endswith('-') else word for word in tmp)
        return pd.Series([name, product, brand])
    
    @staticmethod
    def remove_one_and_two_chars(name: str):
        whitelist = []
        return ' '.join(x.strip() for x in name.split() if len(x) > 2 or x in whitelist)
#         return ' '.join(re.sub(r'\b\w{,2}\b', ' ', word).strip() for word in name.split())
    
    
    def find_brands(self, name, product, brand):
        for br in self.brands:
            if br in name:
                brand = br
                name = name.replace(br, '')
                break
    
        return pd.Series([name, product, brand])
    
    def remove_words_in_blacklist(self, name: str):
        return ' '.join(word for word in name.split() if word.strip() not in self.blacklist)
    
    @staticmethod
    def replace_with_product_dict(name: str):
        return ' '.join(PRODUCTS.get(word.strip(), word) for word in name.split())
        
    def find_scratch_card(self, name, product, brand):
        if 'скретч-карта' in name:
            return pd.Series(['', 'скретч-карта', 'перекресток'])
        return pd.Series([name, product, brand])
    

    @staticmethod
    def find_and_remove_all_english_words(name, brand):
        eng_brands = re.findall(r'\b([a-z]+)\b', name)
        if eng_brands and not brand:
            result = []
            for word in eng_brands:
                name = name.replace(word, '').replace('  ', ' ').strip()
                result.append(word)

            brand = ' '.join(result)
            if 'epica' in brand:
                brand = 'epica'
            return pd.Series([name, brand])

        # If we have brand, we should delete english words in product name
        name = re.sub(r'\b([a-z]+)\b', '', name).replace('  ', ' ').strip()
        return pd.Series([name, brand])
    
    def normalize(self):
        self.df['name_norm'] = self.df['name_norm'].str.lower()
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_all_words_which_contains_number(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_all_words_which_contains_slash(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_punctuation_symbols(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_brands(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df['name_norm'] = self.df['name_norm'].apply(self.remove_one_and_two_chars)
        self.df['name_norm'] = self.df['name_norm'].apply(self.remove_words_in_blacklist)
        self.df['name_norm'] = self.df['name_norm'].apply(self.replace_with_product_dict)
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_scratch_card(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_and_remove_all_english_words(x['name_norm'], x['brand_norm']), axis=1
        )
        return self.df

In [17]:
%%time
n = NormalizeNames(real['name'])
tmp = n.normalize()
# Wall time: 2min 18s

CPU times: user 52.8 s, sys: 46.9 ms, total: 52.8 s
Wall time: 53.3 s


In [18]:
df = pd.concat([real, tmp], axis=1)[['shop_name', 'name', 'name_norm', 'product_norm', 'brand_norm']]

In [19]:
df.brand_norm.dropna().shape # 13020 -> 13719 -> 13726 -> 13778 -> 11344 (без русских брендов)

(11344,)

In [20]:
df.product_norm.dropna().shape # 350 -> 350

(350,)

In [24]:
from typing import Optional, NoReturn, List, Union
from itertools import combinations
from pymystem3 import Mystem


class Finder:
    """
    Searche and recognize the name, category and brand of a product
    from its description.
    Search is carried out in the collected datasets: `rus_brands.csv`,
    `products.csv`, `all_clean.csv`.
    
    Parameters
    ----------
    data_to_parse: Union[pd.DataFrame, str],
        Data in which product information will be recognized.
        If data is a pd.DataFrane, it must contain the following columns:
        ['name_norm', 'product_norm', 'brand_norm'].
        If the data is a string, it should be normalized:
        service characters removed, lowercase letters, etc.
    path_to_brands : str
        Path to the dataset where brand information is collected.
    path_to_products : str
        Path to the dataset, which collects information about
        the product name and their categories.
    path_to_product_db : str
        The path to the dataset, which collects scrapied
        information about products from chain stores.
        
    Attributes
    ----------
    mystem : class
        A Python wrapper of the Yandex Mystem 3.1 morphological
        analyzer (http://api.yandex.ru/mystem).
        See aslo `https://github.com/nlpub/pymystem3`.
    rus_brands : np.ndarray
        List of Russian brands.
    products : pd.DataFrame
        DataFrame of product names and categories.
    df : pd.DataFrame
        The copy of `data_to_parse`.

    Examples
    --------
    TODO
    """ # TODO: Add Examples в доку
    
    def __init__(self, data_to_parse: Union[pd.DataFrame, str], path_to_brands: str = 'clean_data/brands_ru.csv',
                 path_to_products: str = 'clean_data/products.csv', path_to_product_db='clean_data/all_clean.csv'):
        self.mystem = Mystem()
        
        # DataFrames:
        self.rus_brands = pd.read_csv(path_to_brands)['brand'].values
        self.products = pd.read_csv(path_to_products)
        self.product_db = pd.read_csv(path_to_product_db)
        
        columns = ['name_norm', 'product_norm', 'brand_norm']
        if isinstance(data_to_parse, pd.DataFrame):
            self.data = data_to_parse[columns].copy()
        else:
            self.data = pd.DataFrame([[data_to_parse, None, None]], columns=columns)
        self.data['cat_norm'] = None  # Add new column
        
    def find_brands(self, name: str, brand: Optional[str] = None) -> pd.Series:
        """
        Find Russian brands using the dataset `rus_brands.csv`.
        For more accurate recognition, a combination of words in a
        different order is used.

        Parameters
        ----------
        name : str
            Product name.
        brand : str, optional (default=None)
            Product category.

        Returns
        -------
        pd.Series
           pd.Series([name, brand])
        """

        if name and not brand:
            names = set([f'{comb[0]} {comb[1]}' for comb in combinations(name.split(), 2)] + name.split())
            for rus_brand in self.rus_brands:
                if rus_brand in names:
                    name = name.replace(rus_brand, '').replace('  ', ' ').strip()
                    return pd.Series([name, rus_brand])
        return pd.Series([name, brand])

    @staticmethod
    def _remove_duplicate_word(arr: List[str]) -> List[str]:
        """
        Remove duplicates in words when one name is a  continuation
        of another: ['вода', 'вода питьевая'] --> ['вода питьевая'].
        
        Parameters
        ----------
        arr : List[str]
            List description of products in different variants.
        
        Returns
        -------
        arr : List[str]
            List description of products without duplicates.
        """

        if max([len(x.split()) for x in arr]) > 1:
            arr = sorted(arr, key=lambda x: len(x.split()))
            one_words = []
            for product in arr.copy():
                if len(product.split()) == 1:
                    one_words.append(product)
                else:
                    for word in one_words:
                        if word in product:
                            arr.remove(word)
        return arr

    def find_product(self, name: str, product: str, category: Optional[str]=None) -> pd.Series:
        """
        Find products name using the dataset `products.csv`.
        For more accurate recognition, a combination of words in a
        different order is used.

        Parameters
        ----------
        name : str
            Product name.
        product : str
            Product description.
        category : str, optional (default=None)
            Product category.

        Returns
        -------
        pd.Series
           pd.Series([name, product, category]) 
        """

        if name and not product:
            names = pd.DataFrame(set([f'{comb[0]} {comb[1]}' for comb in combinations(name.split(), 2)] + name.split()), columns=['Продукт'])
            merge = self.products.merge(names)
            if len(merge):
                product = ', '.join(self._remove_duplicate_word(merge['Продукт'].values))
                category = merge['Категория'].value_counts().index[0]
        return pd.Series([name, product, category])
    
    @staticmethod
    def remove_dash(name: str, product: str) -> str:
        """
        Remove dash from product description.

        Parameters
        ----------
        name : str
            Product name.
        product : str
            Product description.

        Returns
        -------
        str
           Product name without dashs.
        """

        if name and not product:
            name = name.replace('-', ' ')
        return name
    
    def use_mystem(self, name: str, product: str) -> str:
        """
        Use Yandex pymystem3 library to lemmatize words in product descriptions.
        I tried to use pymorphy, but the recognition quality got worse.

        Parameters
        ----------
        name : str
            Product name.
        product : str
            Product description.
            
        Returns
        -------
        str
            Product description after lemmatization.
        """

        if name and not product:
            name = ''.join(self.mystem.lemmatize(name)[:-1])
        return name
    
    def find_category(self, product: str, category: str) -> pd.Series:
        """
        Find a product category using the dataset `products.csv`.

        Parameters
        ----------
        product : str
            Product description.
        category : str
            Product category.

        Returns
        -------
        pd.Series
           pd.Series([product, category]) 
        """

        if product and not category:
            tmp = self.products[self.products['Продукт'] == product]
            if len(tmp):
                category = tmp['Категория'].values[0]

        return pd.Series([product, category])


    def find_product_by_brand(self, product: str, brand: str, category: str) -> pd.Series:
        """
        If we were able to recognize the product brand,
        but could not recongize the product name,
        we can assign the most common product name for this brand.

        Parameters
        ----------
        product : str
            Product description.
        brand : str
            Product brand.
        category : str
            Product category.

        Returns
        -------
        pd.Series
           pd.Series([product, brand, category]) 
        """

        if brand and not product:
            single_brand_goods = self.product_db[self.product_db['Бренд'] == brand]
            if len(single_brand_goods):
                product = single_brand_goods['Продукт'].value_counts().index[0]
                category = single_brand_goods['Категория'].value_counts().index[0]

        return pd.Series([product, brand, category])
    
    
    def print_logs(self) -> None:
        """
        Print the number of recognized brands,
        categories and names of goods.
        """
        
        _len = len(self.data)
        print(
            "Recognized brands: "
            f"{len(self.data['brand_norm'].dropna())}/{_len}, "
            f"products: {len(self.data['product_norm'].dropna())}/{_len}, "
            f"categories: {len(self.data['cat_norm'].dropna())}/{_len}",
            '-'*60, sep='\n', end='\n\n'
        )
    
    def find_all(self):
        
        print('Before:')
        self.print_logs()
        
        # Find brand:
        print('Find brands...')
        self.data[['name_norm', 'brand_norm']] = self.data.apply(lambda x: self.find_brands(x['name_norm'], x['brand_norm']), axis=1)
        self.print_logs()
        
        # Find product and category:
        print('Find product...')
        self.data[['name_norm', 'product_norm', 'cat_norm']] = self.data.apply(lambda x: self.find_product(x['name_norm'], x['product_norm']), axis=1)
        self.print_logs()
        
        # Remove `-` and the second attempt to find a product:
        print('Remove `-`...')
        self.data['name_norm'] = self.data.apply(lambda x: self.remove_dash(x['name_norm'], x['product_norm']), axis=1)
        self.data[['name_norm', 'product_norm', 'cat_norm']] = self.data.apply(lambda x: self.find_product(x['name_norm'], x['product_norm'], x['cat_norm']), axis=1)
        self.print_logs()
        
        # Use Mystem for lemmatization and the third attempt to find a product:
        print('Lemmatization...')
        self.data['name_norm'] = self.data.apply(lambda x: self.use_mystem(x['name_norm'], x['product_norm']), axis=1)
        self.data[['name_norm', 'product_norm', 'cat_norm']] = self.data.apply(lambda x: self.find_product(x['name_norm'], x['product_norm'], x['cat_norm']), axis=1)
        self.print_logs()
        
        # Find category:
        print('Find category...')
        self.data[['product_norm', 'cat_norm']] = self.data.apply(lambda x: self.find_category(x['product_norm'], x['cat_norm']), axis=1)
        self.print_logs()
        
        # Find product by brand:
        print('Find product by brand...')
        self.data[['product_norm', 'brand_norm', 'cat_norm']] = self.data.apply(lambda x: self.find_product_by_brand(x['product_norm'], x['brand_norm'], x['cat_norm']), axis=1)
        self.print_logs()
        return self.data

In [25]:
%%time
finder = Finder(df)
res = finder.find_all()
res = pd.concat([df['name'], res], axis=1)
# Recognized brands: 13726/32091, products: 350/32091, categories: 0/32091
# ------------------------------------------------------------

# Find brands...
# Recognized brands: 17535/32091, products: 350/32091, categories: 0/32091
# ------------------------------------------------------------

# Find product...
# Recognized brands: 17535/32091, products: 29459/32091, categories: 29109/32091
# ------------------------------------------------------------

# Remove `-`...
# Recognized brands: 17535/32091, products: 29558/32091, categories: 29208/32091
# ------------------------------------------------------------

# Lemmatization...
# Recognized brands: 17535/32091, products: 29924/32091, categories: 29574/32091

# Wall time: 3min 19s

Before:
Recognized brands: 11344/32091, products: 350/32091, categories: 0/32091
------------------------------------------------------------

Find brands...
Recognized brands: 17390/32091, products: 350/32091, categories: 0/32091
------------------------------------------------------------

Find product...
Recognized brands: 17390/32091, products: 29457/32091, categories: 29107/32091
------------------------------------------------------------

Remove `-`...
Recognized brands: 17390/32091, products: 29555/32091, categories: 29205/32091
------------------------------------------------------------

Lemmatization...
Recognized brands: 17390/32091, products: 29909/32091, categories: 29559/32091
------------------------------------------------------------

Find category...
Recognized brands: 17390/32091, products: 29909/32091, categories: 29897/32091
------------------------------------------------------------

Find product by brand...
Recognized brands: 17390/32091, products: 30493/32091,

### TODO:

1. Вынести все собранные словари в отдельные классы
2. Подумать насчёт того, чтобы совместить мой список со списком брендов
3. Переводить всё в .py файлы
4. Настроить красивые логи
5. Добавить скетч карту в общие данные


In [39]:
'КОГ.Я ВЫР.'.lower()

'ког.я выр.'

In [40]:
res[res['brand_norm'].apply(lambda x: not bool(x))].sample(10)

,name,name_norm,product_norm,brand_norm,cat_norm
33176,3672843 Томаты черри ПРЕМИУМ крас.250г,томаты черри,томаты,None,"Овощи, фрукты, ягоды"
1863,ЛАЙМ УПАК.3ШТ,лайм,лайм,None,"Овощи, фрукты, ягоды"
37580,3469420 ХЛ.Д.Пряники КЛАССИЧЕСКИЕ 500г,пряники классические,пряники,None,"Хлеб, сладости, снеки"
15077,12:3368252 Лук ЯЛТИНСКИЙ 1кг,лук ялтинский,лук,None,"Овощи, фрукты, ягоды"
46196,З/П КОМПЛ/ОТБ 100МЛ,компл,зубная паста,None,"Красота, гигиена, бытовая химия"
306,ХЛЕБ КЕЛЬН МНОГОЗЕРНОВОЙ 400Г,хлеб кельн многозерновой,хлеб,None,"Хлеб, сладости, снеки"
45051,КОФЕ МОЛОТЫЙ 250Г,кофе молотый,кофе молотый,None,"Чай, кофе, сахар"
11958,23*4022853 Томаты Черри КОНФЕТТО 250г,томаты черри конфетто,томаты,None,"Овощи, фрукты, ягоды"
14379,10:1536 Лимоны 1кг,лимоны,лимоны,None,"Овощи, фрукты, ягоды"
11599,37:17662 Чеснок 1шт,чеснок,чеснок,None,"Соусы, орехи, консервы"


In [29]:
q['Категория'].unique()

array(['Красота, гигиена, бытовая химия', 'Хлеб, сладости, снеки',
       'Молоко, сыр, яйца', 'Соусы, орехи, консервы',
       'Макароны, крупы, специи', 'Другое', 'Воды, соки, напитки',
       'Рыба, икра', 'Чай, кофе, сахар', 'Замороженные продукты',
       'Птица, мясо, деликатесы', 'Посуда', 'Овощи, фрукты, ягоды',
       'Товары для мам и детей', 'Товары для дома и дачи',
       'Подборки и готовые блюда', 'Зоотовары', 'Бытовая техника',
       'Алкоголь', 'Дача и гриль'], dtype=object)

In [27]:
q = pd.read_csv('clean_data/products.csv')

In [36]:
q[q['Продукт'] == 'сардельки']

,Продукт,Категория
291,сардельки,"Птица, мясо, деликатесы"


In [151]:
q.drop(index=461).to_csv('clean_data/products.csv', index=False)

In [30]:
q.loc[638]['Категория'] = 'Молоко, сыр, яйца'

In [32]:
q.loc[950]

Продукт              халат махровый
Категория    Товары для дома и дачи
Name: 950, dtype: object

In [115]:
q.to_csv('clean_data/products.csv', index=False)

In [284]:
'макароны' in q['Продукт'].values

True

In [60]:
l = ['сгущенка', 'открытка', 'поджарка', 'белок', 'плитка', 'окорочка', 'гвоздика', 'горошек', 'редька', 'рассол', 'пасха', 'кислота лимонная']

In [38]:
q['Категория'].unique()

array(['Красота, гигиена, бытовая химия', 'Хлеб, сладости, снеки',
       'Молоко, сыр, яйца', 'Соусы, орехи, консервы',
       'Макароны, крупы, специи', 'Другое', 'Воды, соки, напитки',
       'Рыба, икра', 'Чай, кофе, сахар', 'Замороженные продукты',
       'Птица, мясо, деликатесы', 'Посуда', 'Овощи, фрукты, ягоды',
       'Товары для мам и детей', 'Товары для дома и дачи',
       'Подборки и готовые блюда', 'Зоотовары', 'Бытовая техника',
       'Алкоголь', 'Дача и гриль'], dtype=object)

In [78]:
tmp = pd.DataFrame.from_dict(white_list, orient='index').reset_index().rename(columns={'index': 'Продукт', 0: 'Категория'})

In [125]:
l = ['яблоко', 'долма', 'разрыхлитель', 'бумага', 'кукуруза']

In [21]:
for i in df.product_norm.dropna().drop_duplicates().values:
    if i not in q['Продукт'].values:
        print(i)

тулетная бумага
шариковая ручка
пучок


In [22]:
tmp = pd.DataFrame(
    [
        ['тулетная бумага', 'Красота, гигиена, бытовая химия'],
        ['шариковая ручка', 'Другое'],
        ['пучок', 'Овощи, фрукты, ягоды'],
    ], columns=['Продукт', 'Категория']
)

In [23]:
pd.concat([q, tmp]).to_csv('clean_data/products.csv', index=False)

In [85]:
q.shape

(1265, 2)

In [88]:
q.drop_duplicates(subset='Продукт').to_csv('clean_data/products.csv', index=False)

In [279]:
from collections import Counter

C = Counter()
def lol(name):
    global C
    for i in name.split():
        C[i]+=1



In [280]:
res[res['product_norm'].apply(lambda x: not bool(x))]['name_norm'].apply(lol);